In [1]:
# Initial imports
import os
import pandas as pd
from dotenv import load_dotenv
from nltk.sentiment.vader import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()
from newsapi import NewsApiClient
from finvizfinance.screener.overview import Overview
import alpaca_trade_api as tradeapi
import datetime

%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np

import pandas_datareader as pdr
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()
import warnings
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

warnings.filterwarnings('ignore')


Bad key "text.kerning_factor" on line 4 in
/Users/freddieeisa/anaconda3/envs/pyvizenv/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
http://github.com/matplotlib/matplotlib/blob/master/matplotlibrc.template
or from the matplotlib source distribution


In [2]:
# Read your api key environment variable
from dotenv import load_dotenv
load_dotenv()
api_key = os.getenv("news_api")
print(api_key)

f147b186a1504298a5fca6370fdcfe78


In [3]:
# Create a newsapi client
newsapi = NewsApiClient(api_key=os.environ["news_api"])

In [4]:
foverview = Overview()

filters_dict = {
    'Performance':'Today Up',
    'Price':"Under $2", 
    'Relative Volume': "Over 5"
}
foverview.set_filter(filters_dict=filters_dict)

tickers = foverview.ScreenerView()
#tickers.sort_values(by='Volume', inplace=True, ascending=False)
tickers.set_index('Ticker', inplace=True )
tickers.head(8)

,Company,Sector,Industry,Country,Market Cap,P/E,Price,Change,Volume
Ticker,,,,,,,,,
NEPT,Neptune Wellness Solutions Inc.,Healthcare,Drug Manufacturers - Specialty & Generic,Canada,253810000.0,None,1.96,0.1667,44087769.0
WRN,Western Copper and Gold Corporation,Basic Materials,Other Industrial Metals & Mining,Canada,229160000.0,None,1.69,0.1818,3358816.0


In [5]:
def get_sentiment(company):
    stock_headlines = ( newsapi.get_everything(
    q=company,
    language='en',
    sort_by='relevancy'))

    sentiments = []

    for articles in stock_headlines['articles']:
        try:
            text = articles['content']
            results = analyzer.polarity_scores(text)
            compound = results['compound']
            pos = results['pos']
            neu = results['neu']
            neg = results['neg'] 

            #put data into sentiments
            sentiments.append({
                'text':text,
                'Compound':compound,
                'Positive':pos,
                'Negative':neg,
                'Neutral':neu})

        except AttributeError:
            pass
    fininal_sentiments = pd.DataFrame(sentiments)
    if not fininal_sentiments.empty:
        return(fininal_sentiments["Positive"].mean())
    else:
        return(0)

In [6]:
ticker_sentiment = [] 
for ticker in tickers.index: 
    sentiment = get_sentiment(tickers['Company'][ticker])
    print(sentiment)
    if float(sentiment) >= 0.05:
        ticker_sentiment.append(ticker)
        print(f"{ticker} does meets our sentiment requirement")
    else:
        print(f"{ticker} does not meet our sentiment requirement")

0.1689285714285714
NEPT does meets our sentiment requirement
0.0511
WRN does meets our sentiment requirement


In [7]:
### Set Alpaca API key and secret
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")

api = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version = "v2"
)

In [8]:
#Set time frame for reporting stocks
today = datetime.date.today()
tomorrow = today + datetime.timedelta(days = 2) 
start_time = tomorrow.strftime("%b %d %Y") + " 10:00AM"
next_day = today + datetime.timedelta(days = 1) 
end_time = next_day.strftime("%b %d %Y") + " 1:00PM"

date_from = datetime.datetime.strptime(
    start_time, '%b %d %Y %I:%M%p')
date_to = datetime.datetime.strptime(
    end_time, '%b %d %Y %I:%M%p')

In [9]:
# Set timeframe to '1D'
timeframe = "1D"

#Set timeframe for last 2 years
start_date = pd.Timestamp("2018-05-01", tz="America/New_York").isoformat()
end_date = pd.Timestamp(next_day, tz="America/New_York").isoformat()

#Get ticker data
df_ticker = api.get_barset(
    ticker_sentiment,
    timeframe,
    start=start_date,
    end=end_date
).df

df_ticker.dropna(inplace=True)
df_ticker.tail()

APIError: internal server error occurred

In [ ]:
#df_ticker=df_ticker.loc[:, (slice(None), ['volume','close'])]
#df_ticker.head()

In [ ]:
df_ticker.columns = ['_'.join(col) for col in df_ticker.columns]
df_ticker.head()

In [ ]:
#df_ticker.iloc[:, [0,2]].plot()
df_ticker.iloc[:, [3,8]].plot()

In [ ]:
df_ticker_1=df_ticker.iloc[:, [0,1,2,3,4]]
df_ticker_1.head()

In [ ]:
import re
delimiters = '_'
matchPattern = ''.join(map(re.escape, delimiters))
df_ticker_1.columns = [re.split(matchPattern, i)[1] for i in df_ticker_1.columns ]
df_ticker_1.head()

In [ ]:
returns_1 = df_ticker_1.iloc[:, [3]].pct_change() * 100
returns_1.plot(title="Returns",legend=False)

In [ ]:
df_ticker_1['returns'] = returns_1.copy()
df_ticker_1 = df_ticker_1.dropna()
df_ticker_1.head()

In [ ]:
# forming classes for classification: -1 for losing day. +1 for winning day
df_ticker_1['target'] = np.where(df_ticker_1.returns.shift(-1) > 0, 1, -1)
df_ticker_1.head()

In [ ]:
# count target values
df_ticker_1['target'].value_counts()

In [ ]:
features = ['volume','returns']
labels = 'target'

In [ ]:
# scatter plot for volume and returns
plt.figure(figsize=(8,3))
sns.scatterplot(x=df_ticker_1['returns'],y=df_ticker_1['volume'])

# Feature Extraction

In [ ]:
# features
features_list = []

# Standard deviation calculation
for i in range(5, 20, 5):
    col_name = 'std_' + str(i)
    df_ticker_1[col_name] = df_ticker_1['close'].rolling(window=i).std()
    features_list.append(col_name)
    
# Mean calculation
for i in range(10, 30, 5):
    col_name = 'ma_' + str(i)
    df_ticker_1[col_name] = df_ticker_1['close'].rolling(window=i).mean()
    features_list.append(col_name)
    
# % change calculation
for i in range(3, 12, 3):
    col_name = 'pct_' + str(i)
    df_ticker_1[col_name] = df_ticker_1['close'].pct_change().rolling(window=i).sum()
    features_list.append(col_name)
    
# volume mean calculation
col_name = 'vma_4'
df_ticker_1[col_name] = df_ticker_1['volume'].rolling(4).mean()
features_list.append(col_name)

# intraday calculation
col_name = 'co'
df_ticker_1[col_name] = df_ticker_1['close'] - df_ticker_1['open']
features_list.append(col_name)

features_list

In [ ]:
# Remove Nans
df_ticker_1.dropna(inplace=True)

#taking a peek at features
df_ticker_1[features_list+['target']].head()

# Create Train_test split

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(df_ticker_1[features_list].iloc[:-1],
                                                   df_ticker_1.iloc[:-1]['target'],
                                                   test_size = 0.2)
X_train.shape, Y_train.shape, X_test.shape, Y_test.shape 

In [ ]:
sns.pairplot(X_train[['std_5','ma_10','vma_4']])

In [ ]:
# scale features
scaler = MinMaxScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
X_train_scaled_df = pd.DataFrame(X_train_scaled, columns=X_train.columns)
sns.pairplot(X_train_scaled_df[['std_5','ma_10','vma_4']])

In [ ]:
X_train_scaled_df.describe().T.round(2)

# Logistic Regression

In [ ]:
# create model
model_1 = LogisticRegression()

# train model
model_1.fit(X_train_scaled, Y_train)

In [ ]:
# make some predictions
Y_pred_train_1 = model_1.predict(X_train_scaled)

Y_pred_1 = model_1.predict(X_test_scaled)

In [ ]:
# accuracy
print("Logistic Regression Model train accuracy:", accuracy_score(Y_train, Y_pred_train_1))
print("Logistic Regression Model test accuracy:", accuracy_score(Y_test, Y_pred_1))


In [ ]:
#plotting confusion matrix
cm = confusion_matrix(Y_test, Y_pred_1)
df = pd.DataFrame(cm, index=['Short', 'Long'], columns=['Short', 'Long'])
plt.figure(figsize=(5,4))
sns.heatmap(df,annot=True, fmt='g')
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title( "Logistic Regression Confusion Matrix", fontsize=15, fontweight="bold")
plt.show()

In [ ]:
#classification report
print(classification_report(Y_test, Y_pred_1))

# Gradient Boosting Classifier

In [ ]:
# create model
model_2 = GradientBoostingClassifier(n_estimators=10, max_features='auto')

# train model
model_2.fit(X_train_scaled, Y_train)

# make some predictions
Y_pred_train_2 = model_2.predict(X_train_scaled)
Y_pred_2 = model_2.predict(X_test_scaled)

# accuracy
print("Gradient Boosting Classifier Model train accuracy:", accuracy_score(Y_train, Y_pred_train_2))
print("Gradient Boosting Classifier Model test accuracy:", accuracy_score(Y_test, Y_pred_2))

In [ ]:
#plotting confusion matrix
cm = confusion_matrix(Y_test, Y_pred_2)
df = pd.DataFrame(cm, index=['Short', 'Long'], columns=['Short', 'Long'])
plt.figure(figsize=(5,4))
sns.heatmap(df,annot=True, fmt='g')
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title( "Gradient Boosting Classifier Confusion Matrix", fontsize=15, fontweight="bold")
plt.show()


In [ ]:
#classification report
print(classification_report(Y_test, Y_pred_2))